In [1]:
#import bibliothèques

import pandas as pd
import numpy as np
from sklearn.cluster import  KMeans


In [2]:
pd.set_option('display.max_columns', None)

df = pd.read_csv('accidents.csv', parse_dates=['date'])

Clustering de la géolocalisation

In [3]:
#corrections coordonnées antilles inversées
idx = df['lat'] < -61
df.loc[idx, ['lat','long']] = df.loc[idx, ['long','lat']].values

coords=df[['lat','long']]

kmeans = KMeans(n_clusters = 60)
df['geoloc'] = kmeans.fit_predict(coords)


Preprocessing final

In [4]:
data = df.drop(['grav','date', 'lat','long'], axis = 1)
#data = df.drop(['grav','date'], axis = 1)

"""
Av => apr => label
1 => 0 => Indemme
4 => 1 => Blessé léger
3 => 2 => Blessé hospitalisé
2 => 3 => Tué
"""
print("Taux avant Modification",df.grav.value_counts(normalize=True)*100)

df['grav']=df['grav'].replace({1:0})
df['grav']=df['grav'].replace({4:1})

df['grav']=df['grav'].replace({3:5})
df['grav']=df['grav'].replace({2:3})
df['grav']=df['grav'].replace({5:2})

print()

print("Taux après Modification",df.grav.value_counts(normalize=True)*100)

target = df['grav']


Taux avant Modification grav
1    41.253169
4    40.312062
3    15.695424
2     2.739346
Name: proportion, dtype: float64

Taux après Modification grav
0    41.253169
1    40.312062
2    15.695424
3     2.739346
Name: proportion, dtype: float64


In [5]:
#différenciation des types d variables
cat_cols = [x for x in data.select_dtypes('int').columns if ( x not in ['nbv','vma','age','hh'] and x[0:4] != 'secu' )]
num_cols = [x for x in data.columns if ( x  in ['nbv','vma','age','hh'] )]

In [6]:
#binarisation des variables catégorielle
data = pd.get_dummies(data = data, columns=cat_cols, dtype='int64', drop_first=True)


In [7]:
data.to_csv('var_explicatives.csv', index = False)
target.to_csv('var_cible.csv', index = False)